<a href="https://colab.research.google.com/github/tpgus2603/2024S-Ajou-ML/blob/main/midterm_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torchprofile
!pip install torch

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [ ]:
import torch
import torchvision.models as models
from torchprofile import profile_macs
import torchvision.transforms as transforms
from torchvision import datasets
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.models.resnet import Bottleneck, ResNet

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(), #증강기법 뒤집기
    transforms.Grayscale(num_output_channels=3),  # 1채널 데이터를 3채널 데이터로 바꾸기 (복잡도를 늘림)
    transforms.RandomRotation(30),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize
])

trainset = datasets.FashionMNIST('MNIST_data/', download=True, train=True, transform=transform)
testset = datasets.FashionMNIST('MNIST_data/', download=True, train=False, transform=transform)

trainloader = DataLoader(trainset, batch_size=128, shuffle=True)
testloader = DataLoader(testset, batch_size=128, shuffle=False)

def count_param(model): #파라미터 측정 함수
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

def calculate_flops(model, input_size): #flops측정함수
    model.eval()
    input = torch.randn(1, 3, input_size, input_size).to(device)
    flops = profile_macs(model, input)
    return flops

def calculate_accuracy(output, target, topk=1): #정확도 측정함수
    with torch.no_grad():
        maxk = topk
        batch_size = target.size(0)
        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))
        correct_k = correct[:maxk].reshape(-1).float().sum(0, keepdim=True)
        accuracy = correct_k.mul_(100.0 / batch_size).item()
        return accuracy

def evaluate_model(model, testloader): #평가함수
    model.eval()
    top1_accuracy = 0
    top5_accuracy = 0

    with torch.no_grad():
        for inputs, labels in testloader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)

            top1 = calculate_accuracy(outputs, labels, topk=1)
            top5 = calculate_accuracy(outputs, labels, topk=5)
            top1_accuracy += top1
            top5_accuracy += top5

    top1_accuracy = top1_accuracy / len(testloader)
    top5_accuracy = top5_accuracy / len(testloader)

    return top1_accuracy, top5_accuracy

criterion = nn.CrossEntropyLoss()

def train_and_evaluate(model, model_name, trainloader, testloader, num_epochs=10, patience=3): #early stopping과 정확도기반 스케쥴러를 위해 학습과 평가를 동시에함
    optimizer = optim.Adam(model.parameters(), lr=0.0005)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.4, patience=3)
    best_accuracy = 0
    best_top5_accuracy = 0
    best_model_state = None
    epochs_no_improve = 0

    for epoch in range(num_epochs):
        model.train()
        for inputs, labels in trainloader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
        print(f'Epoch {epoch + 1}/{num_epochs} completed.')
        current_accuracy, current_top5_accuracy = evaluate_model(model, testloader)
        scheduler.step(current_accuracy)
        print(f"현재 Top-1 정확도: {current_accuracy}")
        print(f"현재 Top-5 정확도: {current_top5_accuracy}")
        if current_accuracy is not None and current_accuracy > best_accuracy:
            best_accuracy = current_accuracy
            best_top5_accuracy = current_top5_accuracy
            best_model_state = model.state_dict()
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
            if epochs_no_improve >= patience:
                print(f'Early stopping at epoch {epoch + 1}')
                break
    if best_model_state is not None:
        model.load_state_dict(best_model_state)
        print(f'{model_name} 최적의 모델로 복원 완료')
    total_params = count_param(model)
    total_flops = calculate_flops(model, 224)
    print(f"총 파라미터 수: {total_params}")
    print(f"총 FLOPs: {total_flops}")
    print(f"최적의 Top-1 정확도: {best_accuracy}")
    print(f"최적의 Top-5 정확도: {best_top5_accuracy}")

    return total_params, best_accuracy

#빠른 학습과 확실한 성능차이를 체감하기위해 파라미터를 대폭 낮춘 가벼워진 rensnet모델 만듬
class LightweightBottleneck(nn.Module):
    expansion = 2

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(LightweightBottleneck, self).__init__()
        width = planes//4  #버틀넥 채널수를 4분의1로 줄임

        self.conv1 = nn.Conv2d(inplanes, width, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(width)
        self.conv2 = nn.Conv2d(width, width, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(width)
        self.conv3 = nn.Conv2d(width, planes * self.expansion, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x): #버틀넥 포워드함수
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out

class LightweightBottleneck101(nn.Module): #resnet101용 버틀넥
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(LightweightBottleneck101, self).__init__()
        width = planes //3

        self.conv1 = nn.Conv2d(inplanes, width, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(width)
        self.conv2 = nn.Conv2d(width, width, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(width)
        self.conv3 = nn.Conv2d(width, planes * self.expansion, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out

class LightweightResNet(nn.Module):   #일반 renset모델 보다 채널(필터)를 절반씩 줄여서 파라미터수를 대폭줄임
    def __init__(self, block, layers, num_classes=10):
        super(LightweightResNet, self).__init__()
        self.inplanes = 32 #featuremap도 모델보다 줄임
        self.conv1 = nn.Conv2d(3, 32, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(32)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 32, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 128, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 256, layers[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(256 * block.expansion, num_classes)

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

class LightweightResNeXtBottleneck(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None, groups=32, base_width=4, dilation=1, norm_layer=None):
        super(LightweightResNeXtBottleneck, self).__init__()
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d

        width = int(planes* (base_width)//64.0) * groups//2

        self.conv1 = nn.Conv2d(inplanes, width, kernel_size=1, bias=False)
        self.bn1 = norm_layer(width)
        self.conv2 = nn.Conv2d(width, width, kernel_size=3, stride=stride, padding=dilation, groups=groups, bias=False, dilation=dilation)
        self.bn2 = norm_layer(width)
        self.conv3 = nn.Conv2d(width, planes * self.expansion, kernel_size=1, bias=False)
        self.bn3 = norm_layer(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out
class LightweightResNeXt(ResNet):
    def __init__(self, block, layers, groups=32, width_per_group=4, num_classes=10):
        super(LightweightResNeXt, self).__init__(block, layers, num_classes=num_classes)
        self.groups = groups
        self.base_width = width_per_group

        self.inplanes = 32 #첫 입력 채널수
        self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # Layers
        self.layer1 = self._make_layer(block, 32, layers[0], groups=groups, width_per_group=width_per_group)
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2, groups=groups, width_per_group=width_per_group)
        self.layer3 = self._make_layer(block, 128, layers[2], stride=2, groups=groups, width_per_group=width_per_group)
        self.layer4 = self._make_layer(block, 256, layers[3], stride=2, groups=groups, width_per_group=width_per_group)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1)) #resnext의 adaptivepool 사용
        self.fc = nn.Linear(256 * block.expansion, num_classes)

    def _make_layer(self, block, planes, blocks, stride=1, dilate=False, groups=32, width_per_group=4, norm_layer=None):
        if norm_layer is None:
            norm_layer = nn.BatchNorm2d

        previous_dilation = self.dilation
        if dilate:
            self.dilation *= stride
            stride = 1

        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion, kernel_size=1, stride=stride, bias=False),
                norm_layer(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample, groups, width_per_group, previous_dilation, norm_layer=norm_layer))
        self.inplanes = planes * block.expansion
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes, groups=groups, base_width=width_per_group, norm_layer=norm_layer, dilation=self.dilation))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x



def lightweight_resnet50():
    return LightweightResNet(LightweightBottleneck, [3, 4, 6, 3])  #인자로 3463을 넣어주면 50layer가 됨

def lightweight_resnet101():
    return LightweightResNet(LightweightBottleneck101, [3, 4, 23, 3]) #인자로 3 4 23 3 을 넣어주면 101layer가 됨

def lightweight_resnext50():
    return LightweightResNeXt(LightweightResNeXtBottleneck, [3, 4, 6, 3])

def lightweight_resnext101():
    return LightweightResNeXt(LightweightResNeXtBottleneck, [3, 4, 23, 3],width_per_group=8)

# Initialize models
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
light_resnet50 = lightweight_resnet50().to(device)
light_resnet101 = lightweight_resnet101().to(device)
light_resnext50 = lightweight_resnext50().to(device)
light_resnext101 = lightweight_resnext101().to(device)

models_list = [(light_resnet50, 'resnet50'),(light_resnext50, 'resnext50'),(light_resnet101, 'resnet101'),(light_resnext101, 'resnext101'),
                ]
for model, model_name in models_list:
    params=count_param(model)
    print(f'{model_name} 파라미터수 :{params}')

target_params = {}

for model, model_name in models_list:
    print(f'{model_name} 학습 및 평가 중...')
    params, accuracy = train_and_evaluate(model, model_name, trainloader, testloader, num_epochs=20, patience=6)
    target_params[model_name] = (params, accuracy)
    print(f'{model_name} 학습 및 평가 완료.')

# Display results
for model_name, (params, accuracy) in target_params.items():
    print(f'{model_name} 파라미터 수: {params}, 최적의 Top-1 정확도: {accuracy}')



resnet50 파라미터수 :783050
resnext50 파라미터수 :858698
resnet101 파라미터수 :3172625
resnext101 파라미터수 :3305194
resnet50 학습 및 평가 중...
Epoch 1/20 completed.
현재 Top-1 정확도: 79.74683544303798
현재 Top-5 정확도: 99.49564873417721
Epoch 2/20 completed.
현재 Top-1 정확도: 85.41337025316456
현재 Top-5 정확도: 99.70332278481013
Epoch 3/20 completed.
현재 Top-1 정확도: 83.38607594936708
현재 Top-5 정확도: 99.75276898734177
Epoch 4/20 completed.
현재 Top-1 정확도: 87.8757911392405
현재 Top-5 정확도: 99.81210443037975
Epoch 5/20 completed.
현재 Top-1 정확도: 86.47151898734177
현재 Top-5 정확도: 99.75276898734177
Epoch 6/20 completed.
현재 Top-1 정확도: 87.29232594936708
현재 Top-5 정확도: 99.74287974683544
Epoch 7/20 completed.
현재 Top-1 정확도: 88.5185917721519
현재 Top-5 정확도: 99.7626582278481
Epoch 8/20 completed.
현재 Top-1 정확도: 69.71914556962025
현재 Top-5 정확도: 96.91455696202532
Epoch 9/20 completed.
현재 Top-1 정확도: 88.8943829113924
현재 Top-5 정확도: 99.82199367088607
Epoch 10/20 completed.
현재 Top-1 정확도: 89.56685126582279
현재 Top-5 정확도: 99.9011075949367
Epoch 11/20 completed.
현

In [ ]:

model=light_resnext101
total_flops = calculate_flops(model, 224)
print(f"resnext_101 총 FLOPs: {total_flops}")

resnext_101 총 FLOPs: 841278720
